# Customer Targeting Analysis: Identifying High-Value Customers for Coca-Cola Campaign

## Business Objective
Identify the **top 100 high-value customers** in the Sugary Drinks category for a targeted Coca-Cola sales campaign.

## Why 100 Customers?
- **Sales capacity**: Typical sales team of 5 reps can manage ~20 key accounts each
- **Pareto principle**: Top customers typically drive disproportionate revenue
- **Campaign focus**: Concentrated effort yields better conversion than broad outreach

## Methodology
1. Data exploration and validation
2. Filter to relevant category (Sugary Drinks)
3. Aggregate customer value using **Sales Amount** (not Quantity)
4. Apply recency filter to ensure customers are still active
5. Analyze concentration and validate selection

### Why Sales Amount over Quantity?
- **Revenue focus**: Campaign ROI is measured in dollars, not units
- **Margin proxy**: Higher-value purchases often indicate premium product mix
- **Customer quality**: Wholesale buyers (high quantity, low margin) less valuable than premium retail

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

## Helper Functions
Reusable functions for customer analysis - can be applied to other categories or campaigns.

In [ ]:
def load_and_validate_data(file_path):
    """
    Load transaction data and perform basic validation.
    
    Args:
        file_path: Path to CSV file
    
    Returns:
        DataFrame with validated data and summary stats
    """
    df = pd.read_csv(file_path)
    
    # Convert date column
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], dayfirst=True)
    
    # Validation checks
    validation = {
        'total_rows': len(df),
        'null_counts': df.isnull().sum().to_dict(),
        'date_range': (df['Transaction Date'].min(), df['Transaction Date'].max()),
        'unique_customers': df['Customer Id'].nunique(),
        'unique_categories': df['Category'].nunique(),
        'negative_sales': (df['Sales Amount'] < 0).sum()
    }
    
    print("=== Data Validation ===")
    print(f"Total transactions: {validation['total_rows']:,}")
    print(f"Date range: {validation['date_range'][0].date()} to {validation['date_range'][1].date()}")
    print(f"Unique customers: {validation['unique_customers']:,}")
    print(f"Categories: {validation['unique_categories']}")
    print(f"Negative sales (returns): {validation['negative_sales']:,}")
    
    if df.isnull().any().any():
        print(f"\nWarning: Null values found:")
        for col, count in validation['null_counts'].items():
            if count > 0:
                print(f"  - {col}: {count:,}")
    else:
        print("\nNo null values found.")
    
    return df, validation


def get_top_customers(df, category, metric='Sales Amount', n=100, 
                      recency_days=None, reference_date=None):
    """
    Identify top N customers by metric within a category.
    
    Args:
        df: Transaction DataFrame
        category: Product category to filter (e.g., 'SUGARY DRINKS')
        metric: Column to rank by (default: 'Sales Amount')
        n: Number of customers to return (default: 100)
        recency_days: Only include customers active within this many days (optional)
        reference_date: Date to calculate recency from (default: max date in data)
    
    Returns:
        DataFrame with top N customers and their metrics
    """
    # Filter to category
    filtered = df[df['Category'] == category].copy()
    
    if len(filtered) == 0:
        raise ValueError(f"No transactions found for category: {category}")
    
    # Apply recency filter if specified
    if recency_days is not None:
        if reference_date is None:
            reference_date = df['Transaction Date'].max()
        cutoff_date = reference_date - timedelta(days=recency_days)
        
        # Get customers with recent activity
        recent_customers = filtered[filtered['Transaction Date'] >= cutoff_date]['Customer Id'].unique()
        filtered = filtered[filtered['Customer Id'].isin(recent_customers)]
        print(f"Recency filter: {len(recent_customers):,} customers active in last {recency_days} days")
    
    # Aggregate by customer
    customer_stats = filtered.groupby(['Customer Id', 'Customer Name']).agg({
        metric: 'sum',
        'Transaction Date': ['min', 'max', 'count'],
        'Invoice ID': 'nunique'
    }).reset_index()
    
    # Flatten column names
    customer_stats.columns = [
        'Customer Id', 'Customer Name', 'Total Sales', 
        'First Purchase', 'Last Purchase', 'Transaction Count', 'Unique Orders'
    ]
    
    # Sort and get top N
    top_customers = customer_stats.nlargest(n, 'Total Sales').reset_index(drop=True)
    
    # Add rank
    top_customers.insert(0, 'Rank', range(1, len(top_customers) + 1))
    
    return top_customers


def analyze_concentration(top_customers, total_category_sales):
    """
    Analyze revenue concentration among top customers.
    
    Args:
        top_customers: DataFrame from get_top_customers()
        total_category_sales: Total sales in the category
    
    Returns:
        Dictionary with concentration metrics
    """
    top_sales = top_customers['Total Sales'].sum()
    
    concentration = {
        'top_100_sales': top_sales,
        'total_category_sales': total_category_sales,
        'top_100_pct': (top_sales / total_category_sales) * 100,
        'top_10_sales': top_customers.head(10)['Total Sales'].sum(),
        'top_10_pct': (top_customers.head(10)['Total Sales'].sum() / total_category_sales) * 100,
        'top_1_sales': top_customers.iloc[0]['Total Sales'],
        'top_1_pct': (top_customers.iloc[0]['Total Sales'] / total_category_sales) * 100,
        'median_top_100': top_customers['Total Sales'].median(),
        'ratio_top_to_100th': top_customers.iloc[0]['Total Sales'] / top_customers.iloc[-1]['Total Sales']
    }
    
    print("=== Revenue Concentration Analysis ===")
    print(f"Top 100 customers: ${concentration['top_100_sales']:,.2f} ({concentration['top_100_pct']:.1f}% of category)")
    print(f"Top 10 customers:  ${concentration['top_10_sales']:,.2f} ({concentration['top_10_pct']:.1f}% of category)")
    print(f"Top 1 customer:    ${concentration['top_1_sales']:,.2f} ({concentration['top_1_pct']:.1f}% of category)")
    print(f"\nMedian in top 100: ${concentration['median_top_100']:,.2f}")
    print(f"Ratio (Top 1 / #100): {concentration['ratio_top_to_100th']:.1f}x")
    
    return concentration

## 1. Load and Validate Data

In [ ]:
df, validation = load_and_validate_data("Data.csv")

In [ ]:
# Examine data structure
df.info()

In [ ]:
df.head()

## 2. Explore Categories

In [ ]:
# Category breakdown
category_summary = df.groupby('Category').agg({
    'Sales Amount': ['sum', 'mean', 'count'],
    'Customer Id': 'nunique'
}).round(2)

category_summary.columns = ['Total Sales', 'Avg Transaction', 'Transaction Count', 'Unique Customers']
category_summary = category_summary.sort_values('Total Sales', ascending=False)
category_summary['% of Total'] = (category_summary['Total Sales'] / category_summary['Total Sales'].sum() * 100).round(1)

print("=== Category Breakdown ===")
print(category_summary.to_string())

### Category Selection Rationale

**Selected: SUGARY DRINKS**

- **Direct relevance**: Coca-Cola's core product line falls squarely in this category
- **Market size**: Verify this is a significant category (check % of total above)
- **Excludes**: ENERGY DRINKS (separate Red Bull-style market), MINERAL WATER (different consumer behavior)

## 3. Identify Target Customers

Applying recency filter of **180 days** to ensure we target active customers, not churned ones.

In [ ]:
# Get top 100 customers with recency filter
CATEGORY = 'SUGARY DRINKS'
TOP_N = 100
RECENCY_DAYS = 180  # 6 months

top_customers = get_top_customers(
    df, 
    category=CATEGORY, 
    n=TOP_N,
    recency_days=RECENCY_DAYS
)

print(f"\nTop {TOP_N} {CATEGORY} customers (active in last {RECENCY_DAYS} days):")
top_customers.head(10)

In [ ]:
# Also get without recency filter for comparison
top_customers_all_time = get_top_customers(
    df, 
    category=CATEGORY, 
    n=TOP_N,
    recency_days=None
)

# How many top all-time customers are still active?
overlap = set(top_customers['Customer Id']).intersection(set(top_customers_all_time['Customer Id']))
print(f"\n{len(overlap)} of top {TOP_N} all-time customers are still active (in last {RECENCY_DAYS} days)")
print(f"{TOP_N - len(overlap)} all-time top customers may have churned")

## 4. Concentration Analysis

Understanding how concentrated revenue is among top customers helps validate our targeting strategy.

In [ ]:
# Calculate total category sales for concentration analysis
total_sugary_drinks_sales = df[df['Category'] == CATEGORY]['Sales Amount'].sum()

concentration = analyze_concentration(top_customers, total_sugary_drinks_sales)

In [ ]:
# Visualize the distribution
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sales distribution
axes[0].bar(top_customers['Rank'], top_customers['Total Sales'], color='steelblue', alpha=0.7)
axes[0].axhline(y=top_customers['Total Sales'].median(), color='red', linestyle='--', label=f'Median: ${top_customers["Total Sales"].median():,.0f}')
axes[0].set_xlabel('Customer Rank')
axes[0].set_ylabel('Total Sales ($)')
axes[0].set_title('Sales Distribution: Top 100 Sugary Drinks Customers')
axes[0].legend()

# Cumulative percentage
cumsum = top_customers['Total Sales'].cumsum()
cumsum_pct = (cumsum / total_sugary_drinks_sales) * 100
axes[1].plot(top_customers['Rank'], cumsum_pct, color='darkgreen', linewidth=2)
axes[1].axhline(y=80, color='red', linestyle='--', alpha=0.5, label='80% threshold')
axes[1].fill_between(top_customers['Rank'], cumsum_pct, alpha=0.3, color='green')
axes[1].set_xlabel('Number of Customers')
axes[1].set_ylabel('Cumulative % of Category Sales')
axes[1].set_title('Pareto Analysis: Customer Revenue Concentration')
axes[1].legend()

plt.tight_layout()
plt.show()

# Find Pareto point
pareto_point = (cumsum_pct >= 80).idxmax() + 1
print(f"\nPareto insight: Top {pareto_point} customers drive 80% of category revenue")

## 5. Outlier Analysis

The top customer spending 100x more than #100 warrants investigation.

In [ ]:
# Investigate top customers - are they legitimate or data anomalies?
top_5 = top_customers.head(5)

print("=== Top 5 Customer Deep Dive ===")
for _, row in top_5.iterrows():
    cust_id = row['Customer Id']
    cust_transactions = df[(df['Customer Id'] == cust_id) & (df['Category'] == CATEGORY)]
    
    print(f"\n#{int(row['Rank'])} - {row['Customer Name']} (ID: {cust_id})")
    print(f"   Total Sales: ${row['Total Sales']:,.2f}")
    print(f"   Transactions: {row['Transaction Count']} | Unique Orders: {row['Unique Orders']}")
    print(f"   Avg per transaction: ${row['Total Sales']/row['Transaction Count']:,.2f}")
    print(f"   Active period: {row['First Purchase'].date()} to {row['Last Purchase'].date()}")
    print(f"   Top brands: {cust_transactions.groupby('Brand')['Sales Amount'].sum().nlargest(3).to_dict()}")

## 6. Export Results

In [ ]:
# Prepare final output
output_df = top_customers[['Rank', 'Customer Id', 'Customer Name', 'Total Sales', 
                           'Last Purchase', 'Transaction Count']].copy()
output_df.columns = ['Rank', 'Customer ID', 'Customer Name', 'Total Sales ($)', 
                     'Last Purchase Date', 'Transaction Count']

# Format for readability
output_df['Total Sales ($)'] = output_df['Total Sales ($)'].round(2)
output_df['Last Purchase Date'] = output_df['Last Purchase Date'].dt.strftime('%Y-%m-%d')

# Save to CSV
output_df.to_csv("Most_Valuable_Customers.csv", index=False)
print(f"Exported {len(output_df)} customers to Most_Valuable_Customers.csv")

# Display final output
output_df.head(10)

In [ ]:
# Summary statistics for the output
print("=== Final Selection Summary ===")
print(f"Customers selected: {len(output_df)}")
print(f"Total value: ${output_df['Total Sales ($)'].sum():,.2f}")
print(f"Value range: ${output_df['Total Sales ($)'].min():,.2f} - ${output_df['Total Sales ($)'].max():,.2f}")
print(f"Median value: ${output_df['Total Sales ($)'].median():,.2f}")
print(f"\nCategory coverage: {concentration['top_100_pct']:.1f}% of Sugary Drinks sales")

## 7. Recommendations

### Campaign Strategy

Based on the concentration analysis:

1. **Tiered approach**: 
   - **Tier 1 (Top 10)**: High-touch, dedicated account managers, custom pricing
   - **Tier 2 (11-50)**: Regular sales calls, volume incentives
   - **Tier 3 (51-100)**: Email campaigns, standard promotions

2. **Risk mitigation**:
   - Top customer concentration is high - losing #1 would significantly impact revenue
   - Consider loyalty programs to reduce churn risk

3. **Growth opportunities**:
   - Customers in "Low CLV & High Frequency" segment may have room to grow
   - Cross-sell other Coca-Cola products to existing heavy sugary drinks buyers

### Success Metrics

Track campaign performance by:
- **Conversion rate**: % of targeted customers who purchase Coca-Cola
- **Average order value**: Did targeting increase transaction sizes?
- **Retention**: Are these customers still active 6 months post-campaign?